# Mandelbox2D in Tensorflow

In [1]:
# imports for simulation
import tensorflow as tf
import numpy as np

# imports for visualization
import PIL.Image
from io import BytesIO
from IPython.display import clear_output, Image, display

# imports for interaction 
from ipywidgets import interact, FloatSlider
import time

sess = tf.InteractiveSession()

In [2]:
class Timer:
    def __init__(self):
        self.perf_start = time.perf_counter()
        self.proc_start = time.process_time()
    def report(self):
        return str(round((time.perf_counter()-self.perf_start),3))
    def reset(self):
        result = self.report()
        self.perf_start = time.perf_counter()
        self.proc_start = time.process_time()
        return result

In [3]:
def DisplayFractal(a, fmt='jpeg'):
  """given a 2D array of escape-time iteration counts, 
     render as an image"""
  ashape = a.shape
  # a_cyclic = (6.28*a/20.0).reshape(list(a.shape)+[1])

  # img = np.concatenate([10+20*np.cos(a_cyclic),
  #                      30+50*np.sin(a_cyclic),
   #                      155-80*np.cos(a_cyclic)], 2)
  atmp = a.reshape(list(a.shape)+[1])
  img = np.concatenate([atmp, atmp, atmp], 2)
  # img[a==a.max()] = 0     # if reach max counts then not diverged, so setting  to 0
  a = img
  print(img.size)
  a = np.uint8(np.clip(a, 0, 255))
  f = BytesIO()
  PIL.Image.fromarray(a).save(f, fmt)
  clear_output(wait = True)
  display(Image(data=f.getvalue()))
  print(" inshape:", ashape)
  # print("  cshape:", a_cyclic.shape)
  print("  cshape:", atmp.shape)
  print("outshape:", img.shape)

In [4]:
tim1 = Timer()
sess.close()
tf.reset_default_graph()
sess = tf.InteractiveSession()
print("sess startup:", tim1.reset())

# numpy setup
nY, nX = np.mgrid[-7.0:7.0:0.020, -7.0:7.0:0.020] 
print(nX.shape, nY.shape)
print("numpy: " + tim1.reset())

X = tf.Variable(nX, tf.float32)
Y = tf.Variable(nY, tf.float32)
Cx = X
Cy = Y
M = tf.Variable(tf.zeros_like(X, tf.float32))
# Zed = tf.constant(0.0, tf.float64)
scale = tf.placeholder(np.float64, shape=())
escape_distance = tf.placeholder(np.float64, shape=())
inner_radius = tf.placeholder(np.float64, shape=())
outer_radius = tf.placeholder(np.float64, shape=())
boxfold = tf.placeholder(np.float64, shape=())
boxfold2 = 2*boxfold
tf.global_variables_initializer().run()
maxiter = 15

for i in range(maxiter):
    # Mandelbox Ballfold:
    #
    # need to either set up conditional ops before modifying, 
    #    or introduce temp variable to hold intermediate half-fold

    # Option2: introducing temp variable
    #tempX1 = tf.where(X>1, 2-X, X)
    #X = tf.where(X<-1, -2-X, tempX1)

    # Option1: setting up conditional ops first
    #   actually in this case would only need to set up LowX, could inline the other, 
    #   but 
    HighX = X > boxfold
    LowX = X < -boxfold
    X = tf.where(HighX, boxfold2-X, X)
    X = tf.where(LowX, -boxfold2-X, X)

    HighY = Y > boxfold
    LowY = Y < -boxfold
    Y = tf.where(HighY, boxfold2-Y, Y)
    Y = tf.where(LowY, -boxfold2-Y, Y)

    # Mandelbox Spherefold
    # getting distance^2
    R2 = X*X + Y*Y
    R = tf.sqrt(R2)
    LowR = R < inner_radius
    MidR = (R >= inner_radius) & (R < outer_radius)
    #  HighR: if R >= 1 do nothing, so don't need conditional for this
    X = tf.where(LowR, X*4, X)
    X = tf.where(MidR, X/R2, X)
    Y = tf.where(LowR, Y*4, Y)
    Y = tf.where(MidR, Y/R2, Y)

    # Mandelbox scale and translate
    X = scale * X + Cx
    Y = scale * Y + Cy

M = tf.sqrt(X*X + Y*Y)
not_diverged = M < escape_distance
not_diverged = tf.cast(not_diverged, tf.float32) * 200

# tf.global_variables_initializer().run()
#B = tf.where(M>=escape_distance, Zed, X)


sess startup: 0.002
(700, 700) (700, 700)
numpy: 0.004


In [6]:
def view_changed(scaling, fold, iradius, oradius, escape): 
    vtime = Timer()
    # result = M.eval(feed_dict = {scale: scaling, escape_distance: zoffset})
    result = not_diverged.eval(feed_dict={scale:scaling, 
                                          boxfold:fold, 
                                          inner_radius:iradius, 
                                          outer_radius:oradius,
                                          escape_distance:escape}) 
    compute_time = vtime.reset()
    print(result.size)
    #result2 = result.reshape(list(result.shape)+[1])
    #print(result2.size)
    DisplayFractal(result)
    display_time = vtime.reset()
    print("compute time: " + compute_time)
    print("display time: " + display_time)
    
interact(view_changed,  
         scaling = FloatSlider(min=-4,max=4,step=0.05,value=2, continuous_update=False), 
         fold = FloatSlider(min=0, max=4, step=0.05, value=1, continuous_update=False), 
         iradius = FloatSlider(min=0, max=2, step=0.05, value=0.5, continuous_update=False), 
         oradius = FloatSlider(min=0.1, max=4, step=0.05, value=1.0, continuous_update=False), 
         escape = FloatSlider(min=1, max=200, step=1, value=100, continuous_update=False),
        )

<function __main__.view_changed>

Filler

Filler

Filler

Filler

Filler

Filler

Filler

Filler

Filler

Filler

Filler

Filler
<br>
<br>
<br>
<br>

Filler

<br>
<br>
<br>
<br>
<br>
<br>


Filler

<br>
<br>

Filler


